# COGS 108 - Capstone Project

## Project links, files, and basic information

### Websites with datasets:
- San Diego Vehicle Stops:  https://data.sandiego.gov/datasets/police-vehicle-stops/
- Dan Diego Population Data:  http://www.city-data.com/city/San-Diego-California.html

### Websites of needed information:
- San Diego police service areas https://www.sandiego.gov/police/services/divisions (vehcle stop data only records the first two digits)
- San Diego zip code map: http://www.city-data.com/zipmaps/San-Diego-California.html

### Names of datasets
#### *Vehicle Stops*
- 'vehicle_stops_2017.csv'
- 'vehicle_stops_2016.csv'
- 'vehicle_stops_2015.csv'
- 'vehicle_stops_2014.csv'

#### *Vehicle Stops Details*
- 'vehicle_stops_search_details_2017.csv'
- 'vehicle_stops_search_details_2016.csv'
- 'vehicle_stops_search_details_2015.csv'
- 'vehicle_stops_search_details_2014.csv'

#### *Files needed to read Vehicle Stops information*
- Race Codes: 'vehicle_stops_race_codes.csv'    
- Title explanations for Vehicle Stops data: 'vehicle_stops_dictionary.csv'
- Title explanations for Vehicle Stops Details data: 'vehicle_stops_search_details_dictionary.csv'
- Possible actions taken when stopped for Vehicle Stops Details data: 'vehicle_stops_search_details_description_list.csv'

In [2]:
# Imports
%matplotlib inline

# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Web scrapping
import sys
!conda install --yes --prefix {sys.prefix} beautifulsoup4

# Data analysis
import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest

import requests
import PyPDF2 as pdf
#import urllib2

Solving environment: ...working... done

# All requested packages already installed.



C:\Users\Gordee\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [ ]:
df_stops = pd.read_csv('vehicle_stops_2017.csv')
df_stops_info = pd.read_csv('vehicle_stops_search_details_2017.csv')

In [ ]:
print(df_stops_info.shape)

In [178]:
zip_list = list()

# Creates a list of all possible zip codes
for i in range(0, 1000):
    if len(str(i)) == 1:
        code = '9200' + str(i)
        zip_list.append(code)
    elif len(str(i)) == 2:
        code = '920' + str(i)
        zip_list.append(code)
    else:
        code = '92' + str(i)
        zip_list.append(code)

In [159]:
file = 'zip_pop_data\\sandag_census_2010_zip_'
file = os.path.relpath(file + str(92101) + '.pdf')
fpdf = pdf.PdfFileReader(file)
text = fpdf.getPage(0).extractText()
index = text.find('Population\nPercent\nTotal Population')
indexEnd = text.find('Source: SANDAG, Current Estimates (2010)\nPopulation by Race')
print(text[index+19:indexEnd-1])

Total Population
35,623
100%
White
20,971
59%
Hispanic
7,600
21%
Black
3,060
9%
Asian
2,565
7%
Two or More
1,058
3%
American Indian
181
1%
Other
102
<1%
Pacific Islander
86
<1%


In [ ]:
import urllib.request
from fpdf import FPDF
import PyPDF2

def download_file(input):
    url = 'http://datasurfer.sandag.org/download/sandag_census_2010_zip_' + input + '.pdf'
    response = urllib.request.urlopen(url)
    file = open(input + '.pdf', 'wb')


In [198]:
import os

currDir = 'zip_pop_data\\'
for currZip in os.listdir('zip_pop_data'):    
    file = os.fsdecode(currZip)
    fpdf = pdf.PdfFileReader(currDir + file)
    page = fpdf.getPage(0).extractText()
    
    # Gets the beginning and end of the data we want
    index = page.find('Population\nPercent\nTotal Population')
    indexEnd = page.find('Source: SANDAG, Current Estimates (2010)\nPopulation by Race')
    text = page[index+19:indexEnd-1]
    text = list(text)
    for index, item in enumerate(text):
        if item == "\n":
            text[index] = '/'

    text = ''.join(text)
    text = text.split('/')
    groups = list()
    percentages = list()
    populations = list()
    cols = ['Group', 'Population', 'Percent']

    for item in text:
        if '%' in item:
            percentages.append(item)
        elif item[0].isnumeric():
            populations.append(item)
        else:
            groups.append(item) 


    p_df = pd.DataFrame(columns = cols)
    p_df['Group'] = groups
    p_df['Population'] = populations
    p_df['Percent'] = percentages
    print(file)
    print(p_df)
    print('\n')


sandag_census_2010_zip_92037.pdf
              Group Population Percent
0  Total Population     38,652    100%
1             White     29,300     76%
2             Asian      4,305     11%
3          Hispanic      3,371      9%
4       Two or More      1,164      3%
5             Black        300      1%
6             Other        136     <1%
7  Pacific Islander         39     <1%
8   American Indian         37     <1%


sandag_census_2010_zip_92101.pdf
              Group Population Percent
0  Total Population     35,623    100%
1             White     20,971     59%
2          Hispanic      7,600     21%
3             Black      3,060      9%
4             Asian      2,565      7%
5       Two or More      1,058      3%
6   American Indian        181      1%
7             Other        102     <1%
8  Pacific Islander         86     <1%


sandag_census_2010_zip_92102.pdf
              Group Population Percent
0  Total Population     42,967    100%
1          Hispanic     26,906     63%


sandag_census_2010_zip_92154.pdf
              Group Population Percent
0  Total Population     79,233    100%
1          Hispanic     52,298     66%
2             Asian     10,322     13%
3             White     10,191     13%
4             Black      3,983      5%
5       Two or More      1,777      2%
6  Pacific Islander        261     <1%
7             Other        248     <1%
8   American Indian        153     <1%


sandag_census_2010_zip_92173.pdf
              Group Population Percent
0  Total Population     29,318    100%
1          Hispanic     27,216     93%
2             White        835      3%
3             Asian        658      2%
4             Black        375      1%
5       Two or More        158      1%
6  Pacific Islander         43     <1%
7   American Indian         17     <1%
8             Other         16     <1%




In [165]:
file = '92118.pdf'
fpdf = pdf.PdfFileReader(file)
text = fpdf.getPage(0).extractText()[827:996]

text = list(text)
#text.replace("\n", ",")

for index, item in enumerate(text):
    if item == "\n":
        text[index] = '/'
        
text = ''.join(text)
text = text.split('/')
#print(text)


groups = list()
percentages = list()
populations = list()
cols = ['Group', 'Population', 'Percent']

for item in text:
    if '%' in item:
        percentages.append(item)
    elif item[0].isnumeric():
        populations.append(item)
    else:
        groups.append(item) 
    

p_df = pd.DataFrame(columns = cols)
p_df['Group'] = groups
p_df['Population'] = populations
p_df['Percent'] = percentages
p_df


,Group,Population,Percent
0,Total Population,"17,041",100%
1,White,"13,677",80%
2,Hispanic,"2,043",12%
3,Asian,476,3%
4,Two or More,464,3%
5,Black,252,1%
6,American Indian,65,<1%
7,Pacific Islander,41,<1%
8,Other,23,<1%
